In [1]:
import asyncio
from functions import *
import time
import ipywidgets as widgets
from ipywidgets import interactive
import matplotlib.ticker as mtick
from datetime import datetime

In [2]:
%matplotlib widget
import matplotlib.pyplot as plt  
import ipympl
import seaborn as sns

plt.rcParams['axes.edgecolor']='#333F4B'
plt.rcParams['axes.linewidth']=0.8
plt.rcParams['xtick.color']='#333F4B'
plt.rcParams['ytick.color']='#333F4B'

plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.sans-serif'] = 'DejaVu Sans'

plt.rcParams.update({'axes.spines.top': False, 'axes.spines.right': False})
sns.set_palette("Set2")

# Docker: getting into the belly of the beast

Docker has become a staple tool of software engineering. However, it is also resource intensive. This project is a thin wrapper around the command `docker stats`: it aims at summarizing docker's usage metrics of the host's resources.

It does so in real time by plotting intuitive graphs to look into the CPU/Memory usage and IO metrics. Ideally, one can filter all of the plots by the name of the docker compose project or - more generally - any string within the name of any of the containers. 

## Initial Static Summary

Here's the first global look to the project in table form. 

In [9]:
def table_summary(docker_project: str): 
    docker_stats = run_and_get_docker_stats()
    docker_pandas_stats = read_pandas_dataframe(docker_stats, docker_project=docker_project)
    
    if docker_project != "all":
        docker_pandas_stats = docker_pandas_stats[docker_pandas_stats['name'].str.contains(docker_project)]

    styled_df = (docker_pandas_stats.
    style.
    format({'memory_percentage': '{:.2%}',
           'cpu_percentange': '{:.2%}'}).
    hide_index().
    bar(color='#FFA07A', vmin=0, subset=['memory_percentage', 'cpu_percentange'], align='zero', vmax=1))
    return styled_df
docker_project = widgets.Text(
    value='all',
    placeholder='all',
    description='Docker-compose project name')
widgets.interact(table_summary, docker_project=docker_project);

interactive(children=(Text(value='all', description='Docker-compose project name', placeholder='all'), Output(…

In [11]:
def plot_mem_usage(docker_project):
    SLEEP = 0.5

    all_tasks = asyncio.all_tasks()
    for task in all_tasks:
        task.cancel()

    fig1, ax = plt.subplots()
    fig1.suptitle("Docker Stats data", fontsize=20)

    docker_stats = run_and_get_docker_stats()
    docker_pandas_stats = read_pandas_dataframe(docker_stats, docker_project)
    
    if docker_project != "all":
        docker_pandas_stats = docker_pandas_stats[docker_pandas_stats['name'].str.contains(docker_project)]
    
    docker_pandas_stats.plot.barh(y=['memory_percentage', 'cpu_percentange'], x = 'name',
                                         ax = ax)
    ax.set_title("Time last updated " + str(datetime.now()))
    ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
    ax.set_xlim([0, 1])
    plt.show(fig1)
    ax.cla()
    async def do():
        while True:
            docker_stats = run_and_get_docker_stats()
            docker_pandas_stats = read_pandas_dataframe(docker_stats, docker_project)
            
            if docker_project != "all":
                docker_pandas_stats = docker_pandas_stats[docker_pandas_stats['name'].str.contains(docker_project)]
            
            ax.cla()
            docker_pandas_stats.plot.barh(y=['memory_percentage', 'cpu_percentange'], x = 'name',
                                         ax = ax)
            ax.xaxis.set_major_formatter(mtick.PercentFormatter(xmax=1))
            ax.set_xlim([0, 1])
            ax.set_title("Time last updated " + str(datetime.now()))    
            plt.draw()
            await asyncio.sleep(SLEEP)


    asyncio.create_task(do());
docker = widgets.Text(
    value='all',
    placeholder='all',
    description='Docker-compose project name')
plot = interactive(plot_mem_usage, docker_project=docker);
display(plot);

interactive(children=(Text(value='all', description='Docker-compose project name', placeholder='all'), Output(…

In [12]:
l = widgets.link((docker, 'value'), (docker_project, 'value'))